# Thoughts
- I could embed the URLs or YouTube links into in embedding space outputting a vector
- I can consider each sentence individually as a classsification problem, predictind a type from each sentence and then averaging all the sentence classes
- Gonna change it so the indiviudal posts are just one long paragraph

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense, Dropout
import tensorflow.keras.layers as layers
from tensorflow.keras import Input
from absl import logging
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import urllib
from custom_gen import DataGenerator
import glob
import random

Using TensorFlow backend.


Here we're going to load the universal sentence encoder which is a encoder created by Google that can encode any Word sentence or paragraph to a 512 dimensional vector.

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embedding_model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return embedding_model(input)

In [ ]:
def get_params(data):
    max_imgs, max_gifs, max_vids, max_words = 0,0,0,0 
    vocab = []
    for i in range(data.shape[0]):
        if len(imgs[i]) > max_imgs:
            max_imgs = len(imgs[i])
        if len(gifs[i]) > max_imgs:
            max_gifs = len(gifs[i])
        if len(vids[i]) > max_imgs:
            max_vids = len(vids[i])
        if len(paragraphs[i].split(' ')) > max_words:
            max_words = len(paragraphs[i].split(' '))        
        for word in paragraphs[i].split(' '):
            if word not in vocab:
                vocab.append(word)
    return max_imgs, max_vids, max_gifs, max_words, len(vocab)

Load data and create reverse and forward map things to the MBTI scores

In [ ]:
# data = pd.read_csv("embeddings_added.csv")
data = pd.read_csv('data/sentence_data.csv')
# to_array = lambda x : np.array( [float(i) for i in x[1:-1].split(',')] )
# data['posts'] = data['posts'].apply(to_array)


header_data = pd.read_csv('data/mbti_1.csv')
types = pd.unique(header_data['type'])
code = {tp:i for i,tp in enumerate(types)}
rev_code  = {i:tp for i,tp in enumerate(types)}
# for tp in types:
#     data['type'] = data['type'].replace(tp,code[tp])
    
### integer mapping using LabelEncoder
# label_encoder = LabelEncoder()
# integer_encoded = label_encoder.fit_transform(data['type'])
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

# ### One hot encoding
# onehot_encoder = OneHotEncoder(sparse=False)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
# # Datasets
# partition = {'train': [], 'validation': ['id-4']}
# labels = {'id-1': 0, 'id-2': 1, 'id-3': 2, 'id-4': 1}

In this section we need to pull out the URLs from the user posts

In [4]:
imgs = {i:[] for i in range(data.shape[0])}
vids = {i:[] for i in range(data.shape[0])}
gifs = {i:[] for i in range(data.shape[0])}
misc = {i:[] for i in range(data.shape[0])}
words = {i:[] for i in range(data.shape[0])}
paragraphs = []

for i, person in enumerate(data['posts']):
    urls_person = [re.findall("(?P<url>https?://[^\s]+)", post) for post in person.split('|||')]
    w = [re.sub("(?P<url>https?://[^\s]+)", '', post) for post in person.split('|||')]
    urls_clean = []
    
    for url, ws in zip(urls_person,w): 
        if url:
            urls_clean.append(url[0])
        if ws:
            words[i].append(re.sub(' +', ' ',ws))
    
    for url in urls_clean:
        if (".jpg" in url) or (".png" in url):
            imgs[i].append(url)
        elif "www.youtube.com" in url:
            vids[i].append(url)
        elif ".gif" in url:
            gifs[i].append(url)
        else:
            misc[i].append(url)
    para = ''
    for sentence in words[i]:
        para += sentence + ' '
    paragraphs.append(para)

In [10]:
# # Reduce logging output.
# logging.set_verbosity(logging.ERROR)

# message_embeddings = embed(paragraphs)

# for i, message_embedding in enumerate(np.array(message_embeddings).tolist()[0:1]):
#   print("Message: {}".format(paragraphs[i]))
#   print("Embedding size: {}".format(len(message_embedding)))
#   message_embedding_snippet = ", ".join((str(x) for x in message_embedding[0:1]))
#   print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

    
# posts = []
# for message in np.array(message_embeddings).tolist():
#     posts.append(message)
# data['USE'] = posts
# data.to_csv("embeddings_added.csv")





# sentence_data = pd.DataFrame()
# sentences = []
# label = []
# for w in range(data.shape[0]):
#     for sen in embed(words[w]):
#         sentences.append( sen.numpy().tolist() )
#     label.extend([data['type'][w]] * len(words[w]))
# sentence_data['posts'] = sentences
# sentence_data['type'] = label
# sentence_data.to_csv('sentence_data.csv')


# for i in range(data['posts'].shape[0]):
#     np.save('data/{}.npy'.format(i),data['posts'].iloc[i])

In [7]:
# encoding = np.zeros((data.shape[0],512))
# data['USE'][0][1:-1]
# encoding = np.array([list(map(float,i[1:-1].split(','))) for i in data['USE']])

In [41]:
encoding = np.stack(data['posts'].values)
X_train, X_test, y_train, y_test = train_test_split(encoding,onehot_encoded, test_size=0.1, random_state=42)

In [47]:
# compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# fit the model
# model.fit(np.expand_dims(X_train, axis=2), y_train, epochs=40, validation_data=(np.expand_dims(X_test, axis=2),y_test),steps_per_epoch=512)

In [65]:
test = embed(["I eat shit for breakfast. Good thing its not vegan"])
rev_code[np.argmax(model.predict(test))]

'INTP'